In [1]:
import tensorflow as tf

In [2]:
tf_test = tf.data.TFRecordDataset('../dataset/tfrecords/mel_spectrogram_windowed_normalised.tfrecords')
tf_test

2022-02-09 09:17:54.618801: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 09:17:54.733665: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 09:17:54.734083: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 09:17:54.736277: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [3]:
for raw_record in tf_test.take(1):
    print(raw_record)

tf.Tensor(b'\n\xc2\xd8\x04\n\xa7\xd8\x04\n\x07feature\x12\x9a\xd8\x04\n\x96\xd8\x04\n\x92\xd8\x04\x08\x01\x12\n\x12\x03\x08\x80\x01\x12\x03\x08\x96\x01"\x80\xd8\x04=_\xbf1\xc8\x86\xe11\xac\xf3\xeb:*_\x07;@W\xe92\x0er\'3\xe7\\\xb0;\x9dSK2\x96<\x013\x90^k1\x99\x104<\xb2\xe6\xbd>\xfd\xc5\xcc=\x05\xe2\xbe=<D=<]"\x07=\xd2b:?\xc4\x98\xaa=dc\x07>1\xdbD?\x9aJa?J}\xee>\xf2\xcb\x96;\xa5\xda\x85>:|m=ql\xa9<dK\x89>:\x81\xa8?\xee\xe8\xea<\xbbw\xd7=M\xf2u?\xa0\x8d4?\xab4\x1d;[\xbe\xb0>_\x89z<\xc4\x8d\xb2<\x81V\xff=o\xc3\x9f=\xf3Z\xf3<\xfbJ\xb8?n\x1e\xca>m\xd8\x85>c\x02\\>\xfd\x85\xfe;P\x803>#\x05\xe8=A2\x1e>\xc5\xc1\xb4?\x91\xb5\xc1@\xd9\xa6\xff?v\xe7/?\xa20\xab>\x84\x8c\xcc=\n\x0b\x10=\xc7\xa4\xf3>\xa7A)=ss\xf5>\x04\xf0\x8e?\xa4\xc0\xd0=r\x11\x9e>\xca\xab\xa0?\xa89\x00>g\xf5Q=\x1fJ\xd9>w5\xf1>\x0b\x83\xf0<\xc3\xc6\xb6<\xa0\x84\x0b@\\\xfa\xfc>\x04\xbe\x82>;}s>\xfe[\xa7>\r\xb9\x1f=Q\xb0\x0e>y\xae\xe6>uJ\xe6@\x13\xc1G>\xe5\xfc\xad=e{\xe0=<\xae\xa7>g#G>\xd0\xd7\xda=\xf7\x00z<\xdf0\x9b=\xfdn\x92?\x90\xb

In [4]:
import numpy as np

features = {
        'feature': tf.io.FixedLenFeature([], tf.string),
        'dialect': tf.io.FixedLenFeature([], tf.string),
    }

def parse_tf_record(serialized):
    parsed_example = tf.io.parse_single_example(serialized=serialized, features=features)
    parsed_example['feature'] = tf.io.parse_tensor(parsed_example['feature'], tf.float32)[:, :, np.newaxis]
    return parsed_example

tf_test_parsed = tf_test.map(parse_tf_record)

In [5]:
import numpy as np

for i in tf_test_parsed.take(1):
    print(i['dialect'])
    print(i['feature'])

tf.Tensor(b'Ngadas', shape=(), dtype=string)
tf.Tensor(
[[[5.56965896e-09]
  [6.56368471e-09]
  [1.80016970e-03]
  ...
  [2.20611900e-01]
  [5.15196383e-01]
  [7.38890767e-01]]

 [[8.17684920e-10]
  [9.63618407e-10]
  [2.64283997e-04]
  ...
  [3.23881656e-02]
  [7.56362900e-02]
  [1.08476996e-01]]

 [[1.28410260e-09]
  [3.69706799e-09]
  [2.02417257e-03]
  ...
  [2.57737994e+00]
  [5.48182309e-01]
  [1.18291564e-01]]

 ...

 [[1.59491465e-11]
  [6.07183531e-12]
  [1.91526979e-05]
  ...
  [2.47445935e-03]
  [4.28777828e-04]
  [3.42490594e-03]]

 [[1.26028517e-12]
  [6.92756508e-13]
  [4.54094425e-06]
  ...
  [1.42068791e-04]
  [1.40367309e-04]
  [2.53594451e-04]]

 [[1.89607681e-14]
  [1.05956613e-14]
  [1.75300073e-07]
  ...
  [7.25279779e-06]
  [1.31245861e-05]
  [5.79931839e-06]]], shape=(128, 150, 1), dtype=float32)


In [6]:
tf_test_parsed

<MapDataset shapes: {dialect: (), feature: <unknown>}, types: {dialect: tf.string, feature: tf.float32}>

In [10]:
tf_test_parsed.cardinality()

<tf.Tensor: shape=(), dtype=int64, numpy=-2>

In [ ]:
train = tf_test_parsed.shuffle(buffer_size=1000).take()